In [1]:
import os
import sys
import glob
import scipy
import matplotlib
import numpy as np
import pandas as pd
import seaborn as sns
from collections import defaultdict
from matplotlib import pyplot as plt
from matplotlib.backends.backend_pdf import PdfPages
from shutil import copyfile

from tqdm import tqdm

%matplotlib inline
sns.set_style('whitegrid')
pd.set_option('display.max_rows', 100)
matplotlib.rcParams['ps.fonttype'] = 42
matplotlib.rcParams['pdf.fonttype'] = 42
pd.set_option('display.max_columns', 100)

import shutil
from IPython.display import display, HTML

from Bio import SeqIO

In [2]:
PROBLEMATIC_SITES_LOCATION = '../data/problematic_sites_sarsCov2.vcf'


# Get a subsample of genomes for major variants

In [83]:
variants_of_concern = {'B.1.1.7' : "UK variant", 'B.1.351': "South Africa variant", 'B.1.427': "CA variant 1", 
                       'B.1.429': "CA variant 2", 'P.1': "Brazil variant 1", 'B.1.526': "NY variant 1", 
                       'B.1.526.1': "NY variant 2", 'B.1.526.2': "NY variant 3", 'P.2': "Brazil variant 2"}

common_us_variants = ['B.1.2', 'B.1', 'B.1.596', 'B.1.1.519']

lineage_data = pd.read_csv('../lineages.metadata.csv') ## Download this file from pangoLEARN

## get lineages with at least 10 genomes
abundant_lineages = lineage_data['lineage'].value_counts()[lineage_data['lineage'].value_counts() >= 10].index
lineage_data = lineage_data[lineage_data.lineage.isin(abundant_lineages)]

## Randomly Subsample really abundant lineages down to 100 genomes
subsampled_lineages = lineage_data.groupby("lineage").sample(100, replace=True).groupby("sequence_name").first()

concern_set = subsampled_lineages[subsampled_lineages.lineage.isin(variants_of_concern)].reset_index() #get a set of variants of concern
background_set = subsampled_lineages[subsampled_lineages.lineage.isin(common_us_variants)].reset_index() #get a set of common background variants

concern_set_names = set(concern_set['sequence_name'].to_list()) # set lookups are really fast
background_set_names = set(background_set['sequence_name'].to_list()) # set lookups are really fast

# Create Pandas DF from the MSA

In [84]:
def load_SNPs_from_MSA(msa_loc, reference_id='hCoV-19/Wuhan/WIV04/2019|EPI_ISL_402124|2019-12-30|China', ignore=set(), include=set()):
    """
    Load SNPs from a multiple sequence alignment
    """
    table = defaultdict(list)
    
    # Load the referece sequence
    print("loading reference")
    reference = None
    for record in SeqIO.parse(msa_loc, 'fasta'):
        if record.id == reference_id:
            reference = record
            break
    if reference is None:
        print("Cannot find sequence {0}".format(reference_id))
        raise Exception()
        
    # Load the rest of the sequences into RAM
    print("loading sequences")
    record_ids = []
    record_sequences = []
    record_SNPs = defaultdict(list)
    for record in tqdm(SeqIO.parse(msa_loc, 'fasta')):
        if record.id.split("|")[0].replace("hCoV-19/","") in include:
            record_ids.append(record.description)
            record_sequences.append(str(record.seq).upper())
        
    # Call SNPs
    print("calling snps")
    print("There are {0} columns".format(len(str(reference.seq).upper())))
    reference_position = 0
    #for bases in tqdm(zip(str(reference.seq).upper(), *record_sequences)):
    for column, ref_base in tqdm(enumerate(str(reference.seq).upper())):
        if ref_base in ['A', 'C', 'G', 'T', '-']:
            if reference_position in ignore:
                reference_position += 1
                continue
            
            bases = [r[column] for r in record_sequences]
            for i, base in enumerate(bases): # loop through the records and this column
                if base in ['A', 'C', 'G', 'T', '-']:
                    if base != ref_base:
                        record_SNPs[record_ids[i]].append((reference_position, column, base))
                else:
                    record_SNPs[record_ids[i]].append((reference_position, column, 'NA'))
            reference_position += 1
        else:
            pass
        
    # Make table
    print("making table")
    table = defaultdict(list)
    for record in record_ids:
        SNPs = record_SNPs[record]
        table['record'].append(record)
        table['SNP_count'].append(len([True for s in SNPs if s[2] != 'NA']))
        table['missing_count'].append(len([True for s in SNPs if s[2] == 'NA']))
        table['SNP_IDs'].append(set(SNPs))
        
    ISdb = pd.DataFrame(table)

    return ISdb

# Load problem sites
problem_sites = pd.read_csv(PROBLEMATIC_SITES_LOCATION, comment="#", sep="\t")
filter_sites = problem_sites.query("FILTER=='mask'")
filter_sites.loc[:,'POS'] = filter_sites['POS'] - 1
filter_sites = set(filter_sites['POS'].tolist())

MSA_LOCATION = '../msa_0405/msa_0405.fasta' #download from GISAID
concern_db = load_SNPs_from_MSA(MSA_LOCATION, ignore=filter_sites, include=concern_set_names)
common_db = load_SNPs_from_MSA(MSA_LOCATION, ignore=filter_sites, include=background_set_names)


/home/alex/.local/lib/python3.8/site-packages/pandas/core/indexing.py:1743: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  isetter(ilocs[0], value)
1782it [00:00, 17788.76it/s]

loading reference
loading sequences


914579it [00:50, 18044.62it/s]
189it [00:00, 1888.94it/s]

calling snps
There are 33620 columns


33620it [00:02, 12198.67it/s]
1816it [00:00, 18158.68it/s]

making table
loading reference
loading sequences


914579it [00:45, 20079.65it/s]
2242it [00:00, 22411.95it/s]

calling snps
There are 33620 columns


33620it [00:01, 26506.43it/s]

making table


# Identify mutations in variants of concern

In [109]:
## get background variants
background_snps = set()

## for each variant of concern
for var in common_us_variants:
    var_set = background_set[background_set.lineage == var]
    snp_count = defaultdict(int)
    total_seqs = 0
    for index, row in common_db.iterrows():
        if row['record'].split("|")[0].replace("hCoV-19/","") in var_set['sequence_name'].to_list(): 
            total_seqs += 1
            for snp in row['SNP_IDs']:
                snp_count[snp] += 1
    print(total_seqs)
    for snp in snp_count:
        if snp_count[snp] >= total_seqs * 0.2:
            background_snps.add(snp)
## get snps > 80% of genomes for this variant

## filter to those not found in common_db

94
87
91
82


In [113]:
## for each variant of concern
for var in variants_of_concern:
    var_set = concern_set[concern_set.lineage == var]
    snp_count = defaultdict(int)
    total_seqs = 0
    for index, row in concern_db.iterrows():
        if row['record'].split("|")[0].replace("hCoV-19/","") in var_set['sequence_name'].to_list(): 
            total_seqs += 1
            for snp in row['SNP_IDs']:
                snp_count[snp] += 1
    abund_snps = set()
    for snp in snp_count:
        if snp_count[snp] >= total_seqs * 0.8:
            abund_snps.add(snp)
    
    ## get snps > 80% of genomes for this variant
    for snp in abund_snps:
        print(var + "\t" + str(snp[1]) + "\t" + str(snp[2]))
## filter to those not found in common_db

B.1.1.7	33501	-
B.1.1.7	33469	-
B.1.1.7	33510	-
B.1.1.7	16084	T
B.1.1.7	33514	-
B.1.1.7	138	-
B.1.1.7	33495	-
B.1.1.7	24149	-
B.1.1.7	136	-
B.1.1.7	31540	-
B.1.1.7	7788	C
B.1.1.7	33493	-
B.1.1.7	162	-
B.1.1.7	166	-
B.1.1.7	33519	-
B.1.1.7	26028	A
B.1.1.7	164	-
B.1.1.7	23859	-
B.1.1.7	145	-
B.1.1.7	33502	-
B.1.1.7	12407	-
B.1.1.7	154	-
B.1.1.7	33477	-
B.1.1.7	33481	-
B.1.1.7	163	-
B.1.1.7	23858	-
B.1.1.7	15776	T
B.1.1.7	33504	-
B.1.1.7	33508	-
B.1.1.7	148	-
B.1.1.7	129	-
B.1.1.7	33490	-
B.1.1.7	25789	G
B.1.1.7	131	-
B.1.1.7	31560	T
B.1.1.7	23869	-
B.1.1.7	25649	A
B.1.1.7	157	-
B.1.1.7	121	-
B.1.1.7	33466	-
B.1.1.7	140	-
B.1.1.7	33497	-
B.1.1.7	33492	-
B.1.1.7	12411	-
B.1.1.7	130	-
B.1.1.7	134	-
B.1.1.7	24150	-
B.1.1.7	31559	C
B.1.1.7	33520	-
B.1.1.7	33472	-
B.1.1.7	33476	-
B.1.1.7	33484	-
B.1.1.7	158	-
B.1.1.7	23853	-
B.1.1.7	31228	T
B.1.1.7	33518	-
B.1.1.7	33499	-
B.1.1.7	143	-
B.1.1.7	124	-
B.1.1.7	133	-
B.1.1.7	122	-
B.1.1.7	126	-
B.1.1.7	33478	-
B.1.1.7	33517	-
B.1.1.7	161	-
B.1.1.7

In [79]:
# common_db.plot.scatter("SNP_count", "missing_count")

In [86]:
concern_db

,record,SNP_count,missing_count,SNP_IDs
0,hCoV-19/USA/NY-PRL-2021_0208_04E11/2021|EPI_IS...,181,22,"{(23970, 23970, T), (33439, 33439, -), (33420,..."
1,hCoV-19/USA/CA-UCSF-UC1008/2020|EPI_ISL_977890...,87,280,"{(24796, 24796, NA), (33510, 33510, -), (33495..."
2,hCoV-19/England/CAMC-FB1175/2021|EPI_ISL_85761...,159,1,"{(33501, 33501, -), (20681, 20681, G), (21534,..."
3,hCoV-19/USA/CA-CDPH-UC591/2020|EPI_ISL_847598|...,87,5,"{(3030, 3030, C), (23492, 23492, T), (33501, 3..."
4,hCoV-19/USA/NMDOH-2021043278/2021|EPI_ISL_9794...,75,2,"{(23492, 23492, T), (33501, 33501, -), (31541,..."
...,...,...,...,...
608,hCoV-19/USA/CA-CZB-17238/2020|EPI_ISL_885085|2...,85,2,"{(3030, 3030, C), (23492, 23492, T), (33501, 3..."
609,hCoV-19/USA/NY-PRL-2021_0125_00D03/2021|EPI_IS...,184,0,"{(33439, 33439, -), (33420, 33420, -), (33424,..."
610,hCoV-19/USA/NY-PRL-2021_0203_01H08/2021|EPI_IS...,145,307,"{(33420, 33420, -), (33510, 33510, -), (29971,..."
611,hCoV-19/Brazil/AM-989/2020|EPI_ISL_833169|2020...,70,0,"{(25789, 25789, G), (3190, 3190, T), (120, 120..."


In [81]:
common_db

,record,SNP_count,missing_count,SNP_IDs
0,hCoV-19/USA/MI-MDHHS-SC23299/2021|EPI_ISL_1008...,109,951,"{(20936, 20936, NA), (30553, 30553, NA), (2297..."
1,hCoV-19/USA/CA-QDX-4302/2021|EPI_ISL_915288|20...,132,0,"{(33501, 33501, -), (33469, 33469, -), (33510,..."
2,hCoV-19/USA/TX-HMH-6909/2020|EPI_ISL_546341|20...,121,1452,"{(10488, 10488, NA), (21718, 21718, NA), (2175..."
3,hCoV-19/England/QEUH-5313BD/2020|EPI_ISL_53216...,116,0,"{(33501, 33501, -), (33469, 33469, -), (33510,..."
4,hCoV-19/USA/NM-CDC-LC0004159/2021|EPI_ISL_8864...,199,0,"{(33439, 33439, -), (33420, 33420, -), (33424,..."
...,...,...,...,...
366,hCoV-19/USA/MD-HP02004/2020|EPI_ISL_981176|202...,127,0,"{(33501, 33501, -), (33469, 33469, -), (33510,..."
367,hCoV-19/USA/TX-DSHS-2372/2021|EPI_ISL_978291|2...,68,586,"{(20936, 20936, NA), (30553, 30553, NA), (2120..."
368,hCoV-19/USA/AL-HGSC-JHKS/2021|EPI_ISL_979507|2...,128,0,"{(33501, 33501, -), (16683, 16683, A), (33469,..."
369,hCoV-19/USA/OH-ODH-SC189769/2020|EPI_ISL_87006...,158,0,"{(33439, 33439, -), (33420, 33420, -), (33424,..."


In [80]:
concern_db

,record,SNP_count,missing_count,SNP_IDs
0,hCoV-19/USA/CA-RD-RA02390/2021|EPI_ISL_1010494...,181,1481,"{(29362, 29362, NA), (32989, 32989, NA), (3263..."
1,hCoV-19/USA/NY-PRL-2021_0208_04E11/2021|EPI_IS...,181,22,"{(23970, 23970, T), (33439, 33439, -), (33420,..."
2,hCoV-19/USA/CA-CZB-19309/2020|EPI_ISL_955699|2...,80,100,"{(24099, 24099, NA), (3030, 3030, C), (23492, ..."
3,hCoV-19/England/QEUH-1033924/2021|EPI_ISL_9239...,432,751,"{(279, 279, -), (471, 471, -), (11859, 11859, ..."
4,hCoV-19/Brazil/AM-IEC-177293/2020|EPI_ISL_9185...,117,0,"{(33501, 33501, -), (23518, 23518, T), (33510,..."
...,...,...,...,...
561,hCoV-19/England/QEUH-C31644/2020|EPI_ISL_73147...,156,0,"{(33501, 33501, -), (33469, 33469, -), (33510,..."
562,hCoV-19/USA/NY-NYCPHL-002785/2021|EPI_ISL_9372...,136,73,"{(23970, 23970, T), (17724, 17724, NA), (33501..."
563,hCoV-19/USA/NY-PRL-2021_0203_01H08/2021|EPI_IS...,145,307,"{(33420, 33420, -), (33510, 33510, -), (29971,..."
564,hCoV-19/USA/CA-CSMC247/2020|EPI_ISL_824655|202...,25,0,"{(25789, 25789, G), (32776, 32776, T), (3030, ..."


# Convert to AA Code